In [1]:
import boto3
from mypy_boto3_rds import RDSClient
from pprint import pprint
import botocore.exceptions


In [2]:
s3_bucket_name = 'lab2023iiitg'
security_group_name = 'web-kush-satyadev'
instance_key_name = 'kush_satyadev'
ec2_role_name = 'portfolio-access'
# security_group_id = 'sg-04af8f61f00d1b8ca'
security_group_id = 'sg-036d12829b059381a'


# Create RDS client for Boto3

In [3]:
database_name = "Portfolio"
database_username = "admin"
database_password = "password"
rds_client: RDSClient = boto3.client("rds")

# Create database instance

In [4]:
try:
    new_instance = rds_client.create_db_instance(
        Engine="MySQL",
        DBInstanceClass="db.t3.micro",
        DBInstanceIdentifier=database_name,
        DBName='Portfolio',
        PubliclyAccessible=True,
        MasterUsername=database_username,
        MasterUserPassword=database_password,
        AllocatedStorage=20,
        VpcSecurityGroupIds=[
            security_group_id
        ],
        DBSubnetGroupName='default-satya',
        Port=3306,
        AvailabilityZone='ap-south-1a', 
        
    )["DBInstance"]
    pprint(new_instance)
except botocore.exceptions.ClientError as error:
    print(error)


An error occurred (DBInstanceAlreadyExists) when calling the CreateDBInstance operation: DB instance already exists


# Access the database instance

In [5]:
creation_waiter = rds_client.get_waiter("db_instance_available")
creation_waiter.wait(DBInstanceIdentifier=database_name)

instance = rds_client.describe_db_instances()["DBInstances"][0]
instance


{'DBInstanceIdentifier': 'portfolio',
 'DBInstanceClass': 'db.t3.micro',
 'Engine': 'mysql',
 'DBInstanceStatus': 'available',
 'MasterUsername': 'admin',
 'DBName': 'Portfolio',
 'Endpoint': {'Address': 'portfolio.cy0nkung7pgs.ap-south-1.rds.amazonaws.com',
  'Port': 3306,
  'HostedZoneId': 'Z2VFMSZA74J7XZ'},
 'AllocatedStorage': 20,
 'InstanceCreateTime': datetime.datetime(2023, 9, 29, 3, 44, 17, 161000, tzinfo=tzutc()),
 'PreferredBackupWindow': '23:58-00:28',
 'BackupRetentionPeriod': 1,
 'DBSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-036d12829b059381a',
   'Status': 'active'}],
 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0',
   'ParameterApplyStatus': 'in-sync'}],
 'AvailabilityZone': 'ap-south-1a',
 'DBSubnetGroup': {'DBSubnetGroupName': 'default-satya',
  'DBSubnetGroupDescription': 'default subnet groups',
  'VpcId': 'vpc-0e7754f6bb0a9ba50',
  'SubnetGroupStatus': 'Complete',
  'Subnets': [{'SubnetIdentifier': 'subnet-0fc56dd261b40

Print DB instance hostname:

In [6]:
host_name = instance["Endpoint"]["Address"]
host_name


'portfolio.cy0nkung7pgs.ap-south-1.rds.amazonaws.com'

In [7]:
ec2 = boto3.resource("ec2")

instances = ec2.create_instances(
    ImageId='ami-0ded8326293d3201b',
    MinCount=1,
    MaxCount=1,
    InstanceType="t2.micro",
    KeyName=instance_key_name,
    SecurityGroupIds=[security_group_id],
    IamInstanceProfile={
        'Name': ec2_role_name
    },
    UserData=open(
        "lab5_2101185.sh"
    ).read(),
)

print(instances)
instance = instances[0]
instance.wait_until_running()
instance.load()

print(instance.public_dns_name)


[ec2.Instance(id='i-07ac9c2094a33bbe6')]


KeyboardInterrupt: 